In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [59]:
def scrape_table(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url,headers = headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    rows = []
    for tr in table.find_all("tr"):
        cols = [td.get_text(strip=True) for td in tr.find_all(["td", "th"])]
        if cols:  
            rows.append(cols)
    df = pd.DataFrame(rows)
    df.columns = ["name", "value"]
    df.loc[0, "value"] = df.loc[0, "name"]
    df["name"] = df["name"].str.replace("^- ", "", regex=True)
    df.loc[0, "name"] = "Tên Cảng"
    df= df.set_index("name").T.reset_index(drop=True)
    return df

In [60]:
def make_unique(cols):
    seen = {}
    new_cols = []
    for col in cols:
        if col in seen:
            seen[col] += 1
            new_cols.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_cols.append(col)
    return new_cols

In [61]:
def transform_port_df(df):
    # 1. Get port name
    port_name = df.loc[df["label"].str.contains("Tên Cảng", case=False, na=False), "value"].iloc[0]

    berths = []
    current_berth = None

    for _, row in df.iterrows():
        label, value = row["label"], row["value"]

        # If label exists but no value → new berth
        if pd.isna(value) or value == "":
            current_berth = {
                "port_name": port_name,
                "berth_name": label.strip(),
                "max_dwt": None,
                "berth_length": None
            }
            berths.append(current_berth)

        # If label has a value → belongs to current berth
        elif current_berth is not None:
            if "DWT" in label or "tấn" in label:
                current_berth["max_dwt"] = value
            elif "chiều dài" in label or "m" in label:
                current_berth["berth_length"] = value


    return pd.DataFrame(berths)

In [62]:
base_url = "https://vimawa.gov.vn/vi/cang-bien?page="
links = []
for page in range(0,2):
    main_url = base_url + str(page)
    res = requests.get(main_url)
    soup = BeautifulSoup(res.text, "html.parser")
    page_links = [a["href"] for a in soup.select("td a")]
    links.extend(page_links)

In [63]:
base_url= "https://vimawa.gov.vn"
all_links=[]
for link in links :
    url = base_url + link
    res_sub = requests.get(url)
    sub_soup = BeautifulSoup(res_sub.text,"html.parser")
    sub_links = [a["href"] for a in sub_soup.select("td a")]
    all_links.extend(sub_links)

In [64]:
cols_keep = [
    "Tên Cảng",
    "Tên đơn vị khai thác cảng",
    "Địa chỉ đơn vị khai thác cảng",
    "Vị trí bến cảng",
    "Số điện thoại liên hệ",
    "Công năng khai thác cảng",
    "Diện tích bến cảng (ha)",
    "Năng lực thông qua của bến cảng (T/năm)",
    "Cơ quan QLNN chuyên ngành hàng hải"
]

cols_not_keep = [
    "name",
    "1. Thông tin cơ bản",
    "2. Thông số kỹ thuật",
    "Năng lực thông qua của bến cảng (T/năm)",
    "Tên đơn vị khai thác cảng",
    "Địa chỉ đơn vị khai thác cảng",
    "Vị trí bến cảng",
    "Số điện thoại liên hệ",
    "Công năng khai thác cảng",
    "Diện tích bến cảng (ha)",
    "Cơ quan QLNN chuyên ngành hàng hải"
]

other_cols =[c for c in test.columns if c not in cols_keep]


all_berth=[]
all_ports=[]
base_url= "https://vimawa.gov.vn"
# loop through all port links
for link in all_links :  
    url = base_url + link
    df = scrape_table(url)  
    df.columns = make_unique(df.columns)

    df.columns = df.columns.str.strip()
    # df = df.loc[:, ~df.columns.duplicated()]

    df_port = df.reindex(columns=cols_keep)   # missing cols will be NaN
    all_ports.append(df_port)

    other_cols =[c for c in df.columns if c not in cols_not_keep]
    berth_df = df[other_cols].T.reset_index()
    berth_df.columns = ["label", "value"]
    berth_df = transform_port_df(berth_df)
    new_berth = berth_df.reindex(columns =berth_df.columns)
    all_berth.append(new_berth)
    
final_df = pd.concat(all_ports, ignore_index=True)
final_berth_df = pd.concat(all_berth, ignore_index = True)


In [66]:
other_cols

['Tên Cảng',
 '1. Thông tin cơ bản:',
 '2. Thông số kỹ thuật:',
 'Cầu cảng số 1',
 '+ Tàu vào cảng lớn nhất (DWT)',
 '+ Kích thước chiều dài cầu cảng (m)']

In [73]:
final_df

final_df = final_df.dropna(subset=["Tên đơn vị khai thác cảng"])
final_berth_df = final_berth_df.dropna(subset=["berth_length"])

In [74]:
(final_berth_df)

,port_name,berth_name,max_dwt,berth_length
0,BẾN CẢNG MỸ THỚI - CẢNG BIỂN AN GIANG,1. Thông tin chung,None,500.000
1,BẾN CẢNG MỸ THỚI - CẢNG BIỂN AN GIANG,Cầu cảng số Mỹ Thới,5.000,106
2,BẾN CẢNG MỸ THỚI - CẢNG BIỂN AN GIANG,Bến phao MT01,10.000,213
3,BẾN CẢNG MỸ THỚI - CẢNG BIỂN AN GIANG,Bến phao MT02,10.000,213
4,BẾN CẢNG MỸ THỚI - CẢNG BIỂN AN GIANG,Bến phao MT03,10.000,213
...,...,...,...,...
495,BẾN CẢNG XĂNG DẦU PHƯỚC KHÁNH - CẢNG BIỂN ĐỒNG...,Cầu cảng,32.000,270
498,BẾN CẢNG LPG HỒNG MỘC - CẢNG BIỂN ĐỒNG NAI,Cầu cảng,3.500,145
501,BẾN CẢNG KHO XĂNG DẦU ĐỒNG THÁP - CẢNG BIỂN ĐỒ...,Cầu cảng số 1,5.000,122
504,BẾN CẢNG ĐỒNG THÁP - CẢNG BIỂN ĐỒNG THÁP,Cầu cảng số 1,3.000,67.5


In [75]:
final_df.to_excel("port.xlsx", index=False)
final_berth_df.to_excel("berth.xlsx", index = False)

In [76]:
final_df

,Tên Cảng,Tên đơn vị khai thác cảng,Địa chỉ đơn vị khai thác cảng,Vị trí bến cảng,Số điện thoại liên hệ,Công năng khai thác cảng,Diện tích bến cảng (ha),Năng lực thông qua của bến cảng (T/năm),Cơ quan QLNN chuyên ngành hàng hải
0,BẾN CẢNG MỸ THỚI - CẢNG BIỂN AN GIANG,Công ty cổ phần An Giang,"Quốc lộ 91, Mỹ Thạnh, Long Xuyên, tỉnh An Giang",NaN,NaN,"Cầu cảng Tổng hợp (Hàng rời, hàng khô…)","1,6",NaN,Cảng vụ Hàng hải An Giang
1,BẾN CẢNG TỔNG HỢP BÌNH DƯƠNG - CẢNG BIỂN ĐỒNG NAI,Công ty cổ phần cảng Bình Dương,"Ấp Ngãi Thắng, xã Bình Thắng, huyện Dĩ An, Tân...","KP Quyết Thắng, p. Bình Thắng, TX Dĩ An, Bình ...",0274 3749 470,"Cầu cảng Tổng hợp (Hàng rời, hàng khô…)",5,270.000,Cảng vụ Hàng hải Đồng Nai
2,BẾN CẢNG PHÚ QUÝ- CẢNG BIỂN BÌNH THUẬN,Ban quản lý cảng Phú Quý,"Xã Tam Thanh, huyện Phú Quý, tỉnh Bình Thuận","Xã Tam Thanh, huyện Phú Quý, tỉnh Bình Thuận",,"Cầu cảng Tổng hợp (Hàng rời, hàng khô…)","4,49",123.900,Cảng vụ Hàng hải Bình Thuận
3,BẾN CẢNG TỔNG HỢP VĨNH TÂN- CẢNG BIỂN BÌNH THUẬN,Công ty cổ phần cảng tổng hợp Vĩnh Tân.,"Tầng 6 Tòa nhà Viettel, Khu dân cư Hùng Vương,...","Xã Vĩnh Tân, huyện Tuy Phong, tỉnh Bình Thuận",(0252) 6266668,"Cầu cảng Tổng hợp (Hàng rời, hàng khô…)","51,52",76.348,Cảng vụ Hàng hải Bình Thuận
5,BẾN CẢNG NINH CHỮ- CẢNG BIỂN CÀ NÁ,Ban quản lý khai thác cảng cá Ninh Thuận,"Khu 5, Phường Đông Hải, TP. Phan Rang, Tháp Ch...","Vùng biển - khu nước thuộc xã Trí Hải, Ninh Hả...",028 3895222,"Cầu cảng Tổng hợp (Hàng rời, hàng khô…),",2,"100,000",Cảng vụ Hàng hải Nha Trang
...,...,...,...,...,...,...,...,...,...
138,BẾN CẢNG XĂNG DẦU PHƯỚC KHÁNH - CẢNG BIỂN ĐỒNG...,CÔNG TY TNHH MTV TM DẦU KHÍ ĐỒNG THÁP,"140 QL30, Phường Mỹ Phú, TP. Cao Lãnh, Đồng Tháp","Phía phải luồng hàng hải Sài Gòn - Vũng Tàu, X...",0277 3851 056,"Cầu cảng hàng lỏng (xăng dầu, khí hóa lỏng, dầ...","1,26",1.000.000,Cảng vụ Hàng hải Đồng Nai
139,BẾN CẢNG LPG HỒNG MỘC - CẢNG BIỂN ĐỒNG NAI,DOANH NGHIỆP TƯ NHÂN THƯƠNG MẠI DỊCH VỤ SX HỒN...,"59/2A ấp Nam Lân, Xã Bà Điểm, Huyện Hóc Môn, T...","Bờ trái sông Lòng Tàu, KCN Ông Kèo, Phước Thái...",028 37176885,"Cầu cảng hàng lỏng (xăng dầu, khí hóa lỏng, dầ...","1,5",61.715,Cảng vụ Hàng hải Đồng Nai
140,BẾN CẢNG KHO XĂNG DẦU ĐỒNG THÁP - CẢNG BIỂN ĐỒ...,CÔNG TY CPTM DẦU KHÍ ĐỒNG THÁP,"140 QL30, Phường Mỹ Phú, TP. Cao Lãnh, Đồng Tháp","Bờ phải sông Tiền, phường 11, thành phố Cao Lã...",0277 3851 056,"Cầu cảng hàng lỏng (xăng dầu, khí hóa lỏng, dầ...",8,826.358,Cảng vụ Hàng hải Đồng Tháp
141,BẾN CẢNG ĐỒNG THÁP - CẢNG BIỂN ĐỒNG THÁP,Chi nhánh Đồng Tháp - Công ty CPVT thủy Tân Cảng,"số 1551, quốc lộ 30, phường 11, thành phố Cao ...","Bờ phải sông Tiền, phường 11, thành phố Cao Lã...",02773 894134,"Cầu cảng Tổng hợp (Hàng rời, hàng khô…)",2.6,229.150,Cảng vụ Hàng hải Đồng Tháp
